# Example to use the custom Container for mono reconstruction

In [117]:
import os

from lstchain.io.containers import DL1ParametersContainer
from lstchain.reco.utils import guess_type
from lstchain.gammalearn import load_model, load_camera_parameters

from ctapipe.utils import get_dataset_path
from ctapipe.io import HDF5TableWriter, HDF5TableReader
from ctapipe.calib import CameraCalibrator
from ctapipe.io import event_source

from astropy import units

import torch

## Options

In [118]:
infile = get_dataset_path('gamma_test_large.simtel.gz')   

dl1_parameters_filename = 'dl1.h5'

allowed_tels = {1} # select LST1 only
max_events = 300 # limit the number of events to analyse in files - None if no limit

cal = CameraCalibrator(r1_product='HESSIOR1Calibrator', extractor_product='NeighbourPeakIntegrator')

## Load the model data

In [119]:
if not os.path.isfile('lst_bootcamp_gl_data.tar.gz'):
    !wget https://gitlab.lapp.in2p3.fr/GammaLearn/GammaLearn/raw/master/share/lst_bootcamp_gl_data.tar.gz
    !tar xvzf lst_bootcamp_gl_data.tar.gz

In [120]:
exps_path = 'lst_bootcamp_gl_data/'
camera_parameters_path = exps_path + 'camera_parameters.h5'

### Regression model

In [ ]:
exp_name = 'regression'
regression_network = load_model(exps_path, exp_name, 100, camera_parameters_path)

### Classification model

In [ ]:
exp_name = 'classification'
classification_network = load_model(exps_path, exp_name, 100, camera_parameters_path)

## R0 to DL2

In [ ]:
dl1_container = DL1ParametersContainer() 

In [123]:
with HDF5TableWriter(filename=dl1_parameters_filename, group_name='events', overwrite=True) as writer:

    source = event_source(infile)
    source.allowed_tels = allowed_tels
    source.max_events = max_events

    for i, event in enumerate(source):
        if i%100==0:
            print(i)
        cal.calibrate(event)

        for telescope_id, dl1 in event.dl1.tel.items():
            tel = event.inst.subarray.tels[telescope_id]
            camera = tel.camera

            image = dl1.image[0]
            peakpos = dl1.peakpos[0]

            data = torch.tensor([image, peakpos], dtype=torch.float).unsqueeze(0)
            prediction = regression_network(data).squeeze(0).detach().numpy()
            particle_prediction = classification_network(data)
            particle = torch.max(particle_prediction, 1)[1]

            ## Fill container ##
            dl1_container.fill_mc(event)
            dl1_container.fill_event_info(event)
            dl1_container.set_mc_core_distance(event, telescope_id)
            dl1_container.mc_type = guess_type(infile)
            dl1_container.set_source_camera_position(event, telescope_id)
            
            event.dl2.energy['gl'].energy = 10**prediction[0] * units.TeV
            event.dl2.shower['gl'].core_x = prediction[1] * units.km
            event.dl2.shower['gl'].core_y = prediction[2] * units.km
            event.dl2.shower['gl'].alt = prediction[3] * units.rad
            event.dl2.shower['gl'].az = prediction[4] * units.rad
            event.dl2.classification['gl'].prediction = particle.item()

            ## Save parameters for later training ##
            writer.write(camera.cam_id, [dl1_container, event.dl2.energy['gl'], event.dl2.shower['gl'],
                                        event.dl2.classification['gl']])
                
                

0
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled


In [ ]:
# The file has been created
!ls -lsh dl1.h5

## Loading hdf5 file is very easy with pandas

In [124]:
import pandas as pd
df = pd.read_hdf(dl1_parameters_filename, key='events/LSTCam')

In [125]:
df

,alt,alt_uncert,average_size,az,az_uncert,core_uncert,core_x,core_y,energy,energy_uncert,...,mc_az,mc_core_distance,mc_core_x,mc_core_y,mc_energy,mc_h_first_int,obs_id,prediction,src_x,src_y
0,69.055863,0.0,0.0,184.045663,0.0,0.0,-1.373474,5.904179,0.025354,-1.0,...,0.046534,174.622923,-51.531204,-106.752579,0.010250,19824.572266,7514,0,-0.503035,0.467968
1,67.183680,0.0,0.0,180.854912,0.0,0.0,143.121541,104.547277,0.052406,-1.0,...,6.277897,239.585294,-211.719864,-78.682312,0.120758,24300.533203,7514,0,-1.323027,-0.057215
2,72.243635,0.0,0.0,180.489265,0.0,0.0,-167.161867,13.042707,0.027688,-1.0,...,6.271463,31.903575,10.729630,56.425159,0.031681,17258.339844,7514,0,0.854726,-0.102846
3,70.177448,0.0,0.0,175.514852,0.0,0.0,-68.544373,163.164288,0.073433,-1.0,...,6.199299,262.288108,24.686068,-193.453491,0.126791,41954.234375,7514,0,0.129366,-0.795008
4,70.978465,0.0,0.0,176.327808,0.0,0.0,-217.457592,-46.652306,0.080128,-1.0,...,6.234098,276.822666,173.315231,263.141388,0.146845,35380.851562,7514,1,0.437997,-0.450242
5,68.838745,0.0,0.0,174.707920,0.0,0.0,-77.798225,71.243554,0.036821,-1.0,...,6.179521,99.050748,40.452858,-13.463385,0.065823,22263.962891,7514,1,-0.579898,-1.053016
6,69.951656,0.0,0.0,182.488940,0.0,0.0,13.818027,-76.176301,0.028875,-1.0,...,6.193979,230.428093,17.040657,-162.431519,0.016580,13893.394531,7514,1,-0.036868,-0.859664
7,70.215875,0.0,0.0,176.329106,0.0,0.0,35.261720,38.215768,0.021937,-1.0,...,6.245500,110.780623,73.603035,124.252159,0.009404,15173.717773,7514,1,0.359682,-0.348327
8,71.770248,0.0,0.0,180.342962,0.0,0.0,-87.133437,219.746038,0.099785,-1.0,...,6.279010,251.544837,52.627186,-175.832092,0.248144,28720.640625,7514,1,0.948764,-0.036260
9,69.232286,0.0,0.0,171.122972,0.0,0.0,61.808124,286.485255,0.373465,-1.0,...,6.135493,275.341046,-117.548683,-192.481934,0.407132,36972.437500,7514,0,-0.294832,-1.465980
